# Aula 4 - ETL - Load

**Objetivos:**

- Aplicar Técnicas de Redução de Dimensão
  - Como constatado no [notebook da aula 2 e 3](./etl_aulas_2_e_3.ipynb), há um desbalanceamento entre as raças dos motoristas - quase 3 vezes mais motoristas brancos. 
  - Aplicar técnicas de undersampling - `easy ensemble`  
  - Analisar novamente utilizando `mutual information` se o atributo mais determinante de ser preso mudou.
- Aplicar Técnicas de Amostragem
    - Aplicar PCA: 
      - Treinar Árvore de Decisão e medir a acurácia 
      - Aplicar o PCA - 
      - Treinar Árvore de Decisão e medir a acurácia 
      - A acurácia mudou?


## Carregando Dataset

## Utilizando Mutual Information no Raw Dataset

## Aplicando Easy Ensemble no Dataset

- Após a aplicação do `easy ensemble`, o atributo mais determinante de ser preso mudou?

## Selecionar Atributos mais Determinantes

## Treinar Uma Árvore de Decisão e Medir a Acurácia

## Aplicar PCA Sobre os Atributos Selecionados Anteriormente

## Treinar Uma Árvore de Decisão Com os K PCAs Mais Relevantes e Medir a Acurácia

- A acurácia melhorou após a aplicação do PCA?